In [ ]:
import json
import pandas as pd
import urllib.request
import PIL
from IPython.display import Markdown
import textwrap

In [ ]:
import google.generativeai as genai

In [ ]:
GOOGLE_API_KEY = ""
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel("gemini-pro-vision")

In [ ]:
df = pd.read_csv('pinterest-fashion-dataset.csv')

In [ ]:
df.iloc[11]['image_url']

In [ ]:
image_url = df['image_url'].iloc[0]

In [ ]:
urllib.request.urlretrieve(image_url, "temp_image.jpg")  # Download the image
image = PIL.Image.open('temp_image.jpg')  # Open the image
response = model.generate_content(image)  # Generate response

In [ ]:
response.text

In [ ]:
image = image.convert('RGB')  # Example conversion to RGB format
image.save('temp_image.jpeg')

In [ ]:
image_url = df.iloc[11]['image_url']
category = df.iloc[11]['category']
response = model.generate_content(
        [f"write a search description for an item from the prespective of customer with categoty:{category} for better filtering and retrieval of an item for an image", image],
        generation_config={"temperature": 0}
    )
response.text

In [ ]:
def add_description(row):
    print(f"Image URL => {row['image_url']}")
    urllib.request.urlretrieve(row['image_url'], "temp_image.jpg")  # Download the image
    image = PIL.Image.open('temp_image.jpg')  # Open the image
    category = row['category']
    response = model.generate_content(
        [f"write a short search description for an item with categoty:{category} for an image, use as much detailing as possible for filtering and personalised recommendation", image],
        generation_config={"temperature": 0}
    )
    print(type(response))
    if response.candidates:
        # Extract text from each part and concatenate
        text_parts = [part.text for part in response.candidates[0].content.parts]
        
        # Concatenate the text from all parts
        concatenated_text = ' '.join(text_parts)
        
        # Print or use the concatenated_text as needed
        print(concatenated_text)
        print("")
        return concatenated_text
    else:
        print("No candidates in the response.")
        return ""

In [ ]:
import time
df['image_description'] = df.apply(add_description, axis=1)

In [ ]:
df

In [ ]:
df.iloc[0]['image_url']

In [ ]:
df['image_description']